In [122]:
import requests 
import re as regex
import json
import pprint as pp # pretty print

import aries_cloudagent.wallet.crypto as crypto
import base64
import base58
import uuid
import jupyter_util as util

## Enter Invite Url:

In [123]:
agent1 = {}
plugin = {}
endpoint = "http://localhost:8150/connections/create-admin-invitation-url"
adminInvitationResponse = requests.post(endpoint)
adminInvitationResponse = adminInvitationResponse.text
agent1['url'] = json.loads(adminInvitationResponse)['invitation_url']
print(agent1['url'])

http://agent1.localhost?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiYzFlYjRhYjUtY2ExOC00NmJlLWFiYzktNTIzZTljZjZjN2E5IiwgInJlY2lwaWVudEtleXMiOiBbIjVOV1phdEw5ZVJyV0FjcDlxZWFVczhRZVB5ZE5FMVRMclVzQ0EyWDJQQWhHIl0sICJzZXJ2aWNlRW5kcG9pbnQiOiAiaHR0cDovL2FnZW50MS5sb2NhbGhvc3QiLCAibGFiZWwiOiAiTWFpbiAoYWRtaW4pIn0=


In [124]:
# Process invite url, delete white spaces
agent1['url'] = agent1['url'].replace(" ", "")
# Regex(substitution) to extract only the invite string from url
agent1['invite_string_b64'] = regex.sub(
           r".*(c\_i\=)", 
           r"", 
           agent1['url'])
# Decoding invite string using base64 decoder
agent1['invite_string'] = base64.b64decode(agent1['invite_string_b64'])
# Converting our invite json string into a dictionary 
agent1['invite'] = json.loads(agent1['invite_string'])

print("Decoded invitation:\n")
pp.pprint(agent1['invite'])

Decoded invitation:

{'@id': 'c1eb4ab5-ca18-46be-abc9-523e9cf6c7a9',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',
 'label': 'Main (admin)',
 'recipientKeys': ['5NWZatL9eRrWAcp9qeaUs8QePydNE1TLrUsCA2X2PAhG'],
 'serviceEndpoint': 'http://agent1.localhost'}


In [125]:
### Generating a did key using aries crypto module
# Create a key pair with random seed 
# Order: 0 - public key, 1 - secret / pritvate key
plugin['keypair'] = crypto.create_keypair()
# it seems to me that did is an encoded sub string of public key
# or maybe just some arbitrary random number
did = plugin['keypair'][0][:16]
# final version of our did
plugin['did'] = base58.b58encode(did).decode("ascii")
# encoding keys to base58
plugin['public_key_b58'] = \
    base58.b58encode(plugin['keypair'][0]).decode("ascii")
plugin['private_key_b58'] = \
    base58.b58encode(plugin['keypair'][1]).decode("ascii")

# print(base58.b58encode(did))
# print(plugin['did'])
print("Private key: \n", plugin['private_key_b58'])
print("\nPublic key: \n", plugin['public_key_b58'])

Private key: 
 2QSciKJ3oH8c9rqHjQX8wNcWYSCuaj2YxUdX9AMEY4gj5RB1SBngnVmFtjSkf4mbcf9PbRGUKJcEmhWPgem83tYS

Public key: 
 FLwc3MDwCAn8Mr8XgmamQkeK1Ds1s8Eb43fXWvGPAoJJ


**Sending a connection request to acapy**

In [126]:
uniqueId = str(uuid.uuid4())
# print(uniqueId)

# our request body
message = {
        "@id":  uniqueId,
        "~transport": {
          "return_route": "all"
        },
        "@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/request",
        "label": "Plugin",
        "connection": {
          "DID": plugin['did'],
          "DIDDoc": {
            "@context": "https://w3id.org/did/v1",
            "id": plugin['did'],
            "publicKey": [{
              "id": plugin['did'] + "#keys-1",
              "type": "Ed25519VerificationKey2018",
              "controller": plugin['did'],
              "publicKeyBase58": plugin['public_key_b58']
            }],
            "service": [{
              "id": plugin['did'] + ";indy",
              "type": "IndyAgent",
              "recipientKeys": plugin['public_key_b58'],
              "serviceEndpoint": ""
            }]
          }
        }
      }

**Encoding/packing the request**

In [127]:
# encoding it with aries crypto function using the key that was
# given to us by aca-py in recipientKeys
decodedAcapyKey = base58.b58decode(agent1['invite']['recipientKeys'][0])
ourPrivateKey = plugin['keypair'][1]
# print(plugin['did'])
encodedMessage = \
    crypto.encode_pack_message(json.dumps(message), [decodedAcapyKey], ourPrivateKey)

encodedMessage = encodedMessage.decode("ascii")

In [128]:
connectionRequestResponse = requests.post(agent1['invite']['serviceEndpoint'], data=encodedMessage)
assert connectionRequestResponse.text != "", "invalid response from acapy"

**Response from aca-py**

In [129]:
   
connectionRequestResponseUnpacked = \
    util.unpackMessage(connectionRequestResponse.text, plugin['keypair'][1])

connectionRequestResponseDict = json.loads(connectionRequestResponseUnpacked[0])

print("Decoded acapy response: \n")
pp.pprint(connectionRequestResponseUnpacked)

Decoded acapy response: 

('{"@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/response", '
 '"@id": "77d7615c-12e8-4432-a490-5d55421769ee", "~thread": {"thid": '
 '"5c32193e-7299-4254-863c-7379a43744de"}, "connection~sig": {"@type": '
 '"did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/signature/1.0/ed25519Sha512_single", '
 '"signature": '
 '"QFrPlYQU5oQZrsx_T1AXXNh3Qwb15P3GPIOcMWUA3dRl2esnxuUY0VQdOq_mf6nUpZP-yLgdi7cJkA9RC3DQDw==", '
 '"sig_data": '
 '"AAAAAF7DyLN7IkRJRCI6ICJYN2hXTWNoUU42dzhERzhubjR6TkFtIiwgIkRJRERvYyI6IHsiQGNvbnRleHQiOiAiaHR0cHM6Ly93M2lkLm9yZy9kaWQvdjEiLCAiaWQiOiAiZGlkOnNvdjpYN2hXTWNoUU42dzhERzhubjR6TkFtIiwgInB1YmxpY0tleSI6IFt7ImlkIjogImRpZDpzb3Y6WDdoV01jaFFONnc4REc4bm40ek5BbSMxIiwgInR5cGUiOiAiRWQyNTUxOVZlcmlmaWNhdGlvbktleTIwMTgiLCAiY29udHJvbGxlciI6ICJkaWQ6c292Olg3aFdNY2hRTjZ3OERHOG5uNHpOQW0iLCAicHVibGljS2V5QmFzZTU4IjogIkhSMWFiZDd3cUtvYm1FZTFLZTdHRUY0NnhYdnpYbmtBaUhONnBXdjNneFZUIn1dLCAiYXV0aGVudGljYXRpb24iOiBbeyJ0eXBlIjogIkVkMjU1MTlTaWduYXR1cmVBdXRoZW50aWNhdGlvbjIwMT

**Unpacking connection data embedded in the response**

In [130]:
sig_data_raw = connectionRequestResponseDict['connection~sig']['sig_data']
#  (this is a hack)replacing artifacts that sometimes happen
sig_data_raw = sig_data_raw.replace("-", "1")
sig_data_raw = sig_data_raw.replace("_", "1")

sig_data_raw = base64.b64decode(sig_data_raw)
# avoid first 8 characters as they are a time signature
sig_data_raw = sig_data_raw[8:]
sig_data_raw = sig_data_raw.decode('ascii')

sig_data = json.loads(sig_data_raw)
pp.pprint(sig_data)

{'DID': 'X7hWMchQN6w8DG8nn4zNAm',
 'DIDDoc': {'@context': 'https://w3id.org/did/v1',
            'authentication': [{'publicKey': 'did:sov:X7hWMchQN6w8DG8nn4zNAm#1',
                                'type': 'Ed25519SignatureAuthentication2018'}],
            'id': 'did:sov:X7hWMchQN6w8DG8nn4zNAm',
            'publicKey': [{'controller': 'did:sov:X7hWMchQN6w8DG8nn4zNAm',
                           'id': 'did:sov:X7hWMchQN6w8DG8nn4zNAm#1',
                           'publicKeyBase58': 'HR1abd7wqKobmEe1Ke7GEF46xXvzXnkAiHN6pWv3gxVT',
                           'type': 'Ed25519VerificationKey2018'}],
            'service': [{'id': 'did:sov:X7hWMchQN6w8DG8nn4zNAm;indy',
                         'priority': 0,
                         'recipientKeys': ['HR1abd7wqKobmEe1Ke7GEF46xXvzXnkAiHN6pWv3gxVT'],
                         'serviceEndpoint': 'http://agent1.localhost',
                         'type': 'IndyAgent'}]}}


**Adding the connection to connection list**

In [131]:
connections = [
    {
        "label": agent1['invite']['label'],
        "DIDDoc": sig_data['DIDDoc'],
        "myKey": plugin['keypair'][1]
    }
]

pp.pprint(connections)

[{'DIDDoc': {'@context': 'https://w3id.org/did/v1',
             'authentication': [{'publicKey': 'did:sov:X7hWMchQN6w8DG8nn4zNAm#1',
                                 'type': 'Ed25519SignatureAuthentication2018'}],
             'id': 'did:sov:X7hWMchQN6w8DG8nn4zNAm',
             'publicKey': [{'controller': 'did:sov:X7hWMchQN6w8DG8nn4zNAm',
                            'id': 'did:sov:X7hWMchQN6w8DG8nn4zNAm#1',
                            'publicKeyBase58': 'HR1abd7wqKobmEe1Ke7GEF46xXvzXnkAiHN6pWv3gxVT',
                            'type': 'Ed25519VerificationKey2018'}],
             'service': [{'id': 'did:sov:X7hWMchQN6w8DG8nn4zNAm;indy',
                          'priority': 0,
                          'recipientKeys': ['HR1abd7wqKobmEe1Ke7GEF46xXvzXnkAiHN6pWv3gxVT'],
                          'serviceEndpoint': 'http://agent1.localhost',
                          'type': 'IndyAgent'}]},
  'label': 'Main (admin)',
  'myKey': b'F;p\xab\xb5\xc3\x8f&\x91\x96\xe3&}n,\x92\xa3L\xaa\xdd\xb

### FEATURE DISCOVERY

In [132]:
message = util.buildMessage(
    "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/discover-features/1.0/query",
    query="*"
)
message = util.packMessage(message, connections[0])
# print(encodedMessage)

endpoint = connections[0]['DIDDoc']['service'][0]['serviceEndpoint']
response = requests.post(endpoint, data=message)
# print(response.text)

responseDecoded = util.unpackMessage(response.text, plugin['keypair'][1])
responseDecoded = json.loads(responseDecoded[0])
pp.pprint(responseDecoded)

{'@id': '50299d75-1d2c-4bed-8a45-bcd235aabf01',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/discover-features/1.0/disclose',
 'protocols': [{'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-static-connections/0.1'},
               {'pid': 'https://didcomm.org/credential-presentation/0.1'},
               {'pid': 'https://didcomm.org/notification/1.0'},
               {'pid': 'https://github.com/hyperledger/aries-toolbox/tree/master/docs/admin-basicmessage/0.1'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/basicmessage/1.0'},
               {'pid': 'https://didcomm.org/connections/1.0'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/credential-issuance/0.1'},
               {'pid': 'https://didcomm.org/discover-features/1.0'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/routing/1.0'},
               {'pid': 'https://didcomm.org/routing/1.0'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-connections/0.1'},
           

## Dids

In [133]:
message = util.buildMessage("did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/get-list-dids")
encodedMessage = util.packMessage(message, connections[0])
# print(encodedMessage)

endpoint = connections[0]['DIDDoc']['service'][0]['serviceEndpoint']
response = requests.post(endpoint, data=encodedMessage)
# print(response.text)

responseDecoded = util.unpackMessage(response.text, plugin['keypair'][1])
responseDecoded = json.loads(responseDecoded[0])
pp.pprint(responseDecoded)

{'@id': '9405aa44-a192-487a-bce5-351e857f2339',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/list-dids',
 'result': [{'did': 'WTSnQMUdQjdsxYkKUBTcYJ',
             'verkey': 'H4AJNxyBHdmgENQsME6dhXS345NLVqcvvPM7QtF2X3b5'},
            {'did': 'WjVuoNs4Zc1mK6vXPfqnj5',
             'verkey': 'HCuoA2ewtHWQ297KgJG22VrexYmEbZRCi4aqPkE2cEt9'},
            {'did': 'CcPNApHoaNaxHtrDJx3Fcu',
             'verkey': '7L23hAKHoiLQ4KyaTEkpzi5eoLXt3cLuteFYp6CzDxXk'},
            {'did': 'AH7bQLKyijZkniHvRoLAzu',
             'verkey': '64HqNtZaju4Y5Vd4ZzzBZ4apZqe69gc6AtF9zHr5S9dr'},
            {'did': 'TUNT4Z7kKeYquhmeCZjg8F',
             'verkey': 'FRq4sMiB64yErvd4Rs7k4dmKxzBBVkbFh4Rt6QGmXmbZ'},
            {'did': 'X7hWMchQN6w8DG8nn4zNAm',
             'verkey': 'HR1abd7wqKobmEe1Ke7GEF46xXvzXnkAiHN6pWv3gxVT'},
            {'did': 'B2xyyghsCnQ2svVzD6uce5',
             'verkey': '6UC4vdz81bf5ZCgQk2DoQho3d8jMxmsRbC5StZHHwtc4'},
            {'did': 'WZndzdvhBc6vXp8hFPrCdL',
      

## Public did

In [134]:
message = util.buildMessage("did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/get-public-did")
encodedMessage = util.packMessage(message, connections[0])
# print(encodedMessage)

endpoint = connections[0]['DIDDoc']['service'][0]['serviceEndpoint']
response = requests.post(endpoint, data=encodedMessage)
# print(response.text)

responseDecoded = util.unpackMessage(response.text, plugin['keypair'][1])
responseDecoded = json.loads(responseDecoded[0])
pp.pprint(responseDecoded)

{'@id': 'd31cf65a-c6ab-421d-a9a9-3fd0360a148e',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/did',
 '~thread': {'thid': 'f7619529-1c43-4104-8137-b60e217a3b82'}}


In [135]:
message = util.buildMessage(
    "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/acapy-plugin/1.0/schema",
    payload="a very cool message"
)

encodedMessage = util.packMessage(message, connections[0])
# print(encodedMessage)

endpoint = connections[0]['DIDDoc']['service'][0]['serviceEndpoint']
response = requests.post(endpoint, data=encodedMessage)
# print(response.text)

responseDecoded = util.unpackMessage(response.text, plugin['keypair'][1])
responseDecoded = json.loads(responseDecoded[0])
pp.pprint(responseDecoded)

{'@id': 'ddf493d3-fff6-4100-956f-7d8240c90b0c',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/acapy-plugin/1.0/schema',
 'payload': 'a very cool message',
 'schema_id': '622917b8b7de4b84a3dbc870827fa307',
 '~thread': {'thid': '432fc924-e51c-4be4-9484-4bf7d2d8f67e'}}


In [140]:
message = util.buildMessage(
    "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/acapy-plugin/1.0/search_schema",
    schema_id="6f1b20eaa0a144e6aa5744fe6eaabede"
)

encodedMessage = util.packMessage(message, connections[0])

endpoint = connections[0]['DIDDoc']['service'][0]['serviceEndpoint']
response = requests.post(endpoint, data=encodedMessage)

responseDecoded = util.unpackMessage(response.text, plugin['keypair'][1])
responseDecoded = json.loads(responseDecoded[0])
pp.pprint(responseDecoded)

{'@id': '09a5e950-7544-4c4b-9ba4-da79602674a9',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/acapy-plugin/1.0/schema',
 'payload': 'a very cool message',
 'schema_id': '6f1b20eaa0a144e6aa5744fe6eaabede',
 '~thread': {'thid': '0d5a745a-0721-43a3-8170-ce963829750e'}}
